In [19]:
#Dependencies
import warnings
warnings.simplefilter('ignore')
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [20]:
#Reading the data
data = pd.read_csv("Resources/cleaned_Kepler.csv")
data.head()

,Unnamed: 0,koi_pdisposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,...,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,0,CANDIDATE,0,0,0,0,9.488036,170.538750,0.146,2.95750,...,2.26,793.0,93.59,35.8,5455.0,4.467,0.927,291.93423,48.141651,15.347
1,1,CANDIDATE,0,0,0,0,54.418383,162.513840,0.586,4.50700,...,2.83,443.0,9.11,25.8,5455.0,4.467,0.927,291.93423,48.141651,15.347
2,2,FALSE POSITIVE,0,1,0,0,19.899140,175.850252,0.969,1.78220,...,14.60,638.0,39.30,76.3,5853.0,4.544,0.868,297.00482,48.134129,15.436
3,3,FALSE POSITIVE,0,1,0,0,1.736952,170.307565,1.276,2.40641,...,33.46,1395.0,891.96,505.6,5805.0,4.564,0.791,285.53461,48.285210,15.597
4,4,CANDIDATE,0,0,0,0,2.525592,171.595550,0.701,1.65450,...,2.75,1406.0,926.16,40.9,6031.0,4.438,1.046,288.75488,48.226200,15.509


In [21]:
#Preprocessing the data, getting the binary labels
data_Kepler = pd.get_dummies(data, columns=["koi_pdisposition"])

data_Kepler.head()

,Unnamed: 0,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,...,koi_insol,koi_model_snr,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag,koi_pdisposition_CANDIDATE,koi_pdisposition_FALSE POSITIVE
0,0,0,0,0,0,9.488036,170.538750,0.146,2.95750,615.8,...,93.59,35.8,5455.0,4.467,0.927,291.93423,48.141651,15.347,1,0
1,1,0,0,0,0,54.418383,162.513840,0.586,4.50700,874.8,...,9.11,25.8,5455.0,4.467,0.927,291.93423,48.141651,15.347,1,0
2,2,0,1,0,0,19.899140,175.850252,0.969,1.78220,10829.0,...,39.30,76.3,5853.0,4.544,0.868,297.00482,48.134129,15.436,0,1
3,3,0,1,0,0,1.736952,170.307565,1.276,2.40641,8079.2,...,891.96,505.6,5805.0,4.564,0.791,285.53461,48.285210,15.597,0,1
4,4,0,0,0,0,2.525592,171.595550,0.701,1.65450,603.3,...,926.16,40.9,6031.0,4.438,1.046,288.75488,48.226200,15.509,1,0


In [22]:
#Spliting the data into labels and features
X = data_Kepler.drop(columns=["koi_pdisposition_CANDIDATE", "koi_pdisposition_FALSE POSITIVE", "Unnamed: 0"], axis=1)
y = data_Kepler["koi_pdisposition_CANDIDATE"]
print(X.shape, y.shape)

(9200, 19) (9200,)


In [23]:
#Spliting the data into training and testing parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.4, random_state=42)

In [24]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [25]:
#Creating the model
model = XGBClassifier()

In [26]:
#Training
model.fit(X_train_scaled, y_train)

[22:47:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [27]:
#Evaluating the model
predictions = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, predictions)
print(accuracy)

0.9896739130434783
